In [5]:
ROOT = '/home/anhphantq/otto'
max_session = 64


In [23]:
DATA_FOLDER = ROOT + '/splitted_data'
import pandas as pd
test_df = pd.read_parquet(DATA_FOLDER + '/test.parquet')
train_df = pd.read_parquet(DATA_FOLDER + '/train.parquet')


In [6]:
DATA_FOLDER = ROOT + '/data'
import pandas as pd
test_df = pd.read_parquet(DATA_FOLDER + '/test.parquet')
train_df = pd.read_parquet(DATA_FOLDER + '/train.parquet')


In [ ]:
import pandas as pd
test_df = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/splitted_unidata_sub/test_v1.parquet')
train_df = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/splitted_unidata_sub/train_v1.parquet')


In [2]:
import pandas as pd
test_df = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/unidata_sub/test_v1.parquet')
train_df = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/unidata_sub/train_v1.parquet')


#Add subsession

In [ ]:
import numpy as np

def add_diff(df):
  df = df.sort_values(['session', 'ts'])
  df['diff'] = df['ts'].diff()
  session = df['session'].values
  ends_idx = session[1: ] - session[:-1]
  start_session = np.argwhere(ends_idx > 0) + 1
  start_session  = np.vstack(([0], start_session))
  df.iloc[start_session, -1] = 0
  return df
  

In [ ]:
train_df = add_diff(train_df)
train_df

,session,aid,ts,type,diff
0,0,1517085,1659304800,0,0.0
1,0,1563459,1659304904,0,104.0
2,0,1309446,1659367439,0,62535.0
3,0,16246,1659367719,0,280.0
4,0,1781822,1659367871,0,152.0
...,...,...,...,...,...
216716091,12899776,1737908,1661723987,0,15.0
216716092,12899777,384045,1661723976,0,0.0
216716093,12899777,384045,1661723986,0,10.0
216716094,12899778,561560,1661723983,0,0.0


In [ ]:
test_df = add_diff(test_df)
test_df

,session,aid,ts,type,diff
0,12899779,59625,1661724000,0,0.0
1,12899780,1142000,1661724000,0,0.0
2,12899780,582732,1661724058,0,58.0
3,12899780,973453,1661724109,0,51.0
4,12899780,736515,1661724136,0,27.0
...,...,...,...,...,...
6928118,14571577,1141710,1662328774,0,0.0
6928119,14571578,519105,1662328775,0,0.0
6928120,14571579,739876,1662328775,0,0.0
6928121,14571580,202353,1662328781,0,0.0


In [ ]:
from tqdm.notebook import tqdm

def add_subsession(df, drop = False):
  cur_session = -1
  cur_subsession = -1
  subsessions = []
  max_afk = 0.5 * 3600

  count = 0

  remove = []
  for row in tqdm(df.itertuples(), total = df.shape[0]):
    if row.session != cur_session:

      if cur_subsession != -1 and cur_subsession >= max_session:
        session_tracking = np.array(session_tracking)
        m = np.max(session_tracking)
        upto_remove = m - max_session
        num_redundant = np.sum(np.where(session_tracking <= upto_remove, 1, 0))
        remove += np.arange(start_idx, start_idx + num_redundant).tolist()
      
      if count > 39:
        subsession_tracking = np.array(subsession_tracking)
        m = np.max(subsession_tracking)
        upto_remove = m - 9
        num_redundant = np.sum(np.where(subsession_tracking <= upto_remove, 1, 0))
        remove += np.arange(sub_start_idx, sub_start_idx + num_redundant).tolist()

      #reset sesioon
      cur_session = row.session
      cur_subsession = 0
      
      start_idx = row[0]
      session_tracking = []

      #reset subsession
      sub_start_idx = row[0]
      count = 0 
      subsession_tracking = []





    if row.diff > max_afk:
      if count > 39:
        subsession_tracking = np.array(subsession_tracking)
        m = np.max(subsession_tracking)
        upto_remove = m - 9
        num_redundant = np.sum(np.where(subsession_tracking <= upto_remove, 1, 0))
        remove += np.arange(sub_start_idx, sub_start_idx + num_redundant).tolist()
      
      #reset subsession
      sub_start_idx = row[0]
      count = 0
      subsession_tracking = []
      cur_subsession += 1


    else:
      
      count += 1
    
    subsession_tracking.append(count)
    session_tracking.append(cur_subsession)
    subsessions.append(cur_subsession)



  df['subsession'] = subsessions
  if drop:
    print(f'Delete {len(remove)} rows')
    df = df.drop(index = remove, axis = 0)

  del subsessions
  return df

In [ ]:
train_df = add_subsession(train_df, drop = True)
train_df

  0%|          | 0/216716096 [00:00<?, ?it/s]

Delete 20387744 rows


,session,aid,ts,type,diff,subsession
0,0,1517085,1659304800,0,0.0,0
1,0,1563459,1659304904,0,104.0,0
2,0,1309446,1659367439,0,62535.0,1
3,0,16246,1659367719,0,280.0,1
4,0,1781822,1659367871,0,152.0,1
...,...,...,...,...,...,...
216716091,12899776,1737908,1661723987,0,15.0,0
216716092,12899777,384045,1661723976,0,0.0,0
216716093,12899777,384045,1661723986,0,10.0,0
216716094,12899778,561560,1661723983,0,0.0,0


In [ ]:
test_df = add_subsession(test_df, drop = False)
test_df

  0%|          | 0/6928123 [00:00<?, ?it/s]

,session,aid,ts,type,diff,subsession
0,12899779,59625,1661724000,0,0.0,0
1,12899780,1142000,1661724000,0,0.0,0
2,12899780,582732,1661724058,0,58.0,0
3,12899780,973453,1661724109,0,51.0,0
4,12899780,736515,1661724136,0,27.0,0
...,...,...,...,...,...,...
6928118,14571577,1141710,1662328774,0,0.0,0
6928119,14571578,519105,1662328775,0,0.0,0
6928120,14571579,739876,1662328775,0,0.0,0
6928121,14571580,202353,1662328781,0,0.0,0


# Save df

In [ ]:
train_df.to_parquet('/content/drive/MyDrive/ColabNotebooks/splitted_unidata_sub/train_v2.parquet', index = False)

In [ ]:
train_df.to_parquet('/content/drive/MyDrive/ColabNotebooks/unidata_sub/train_v1.parquet', index = False)

In [ ]:
test_df.to_parquet('/content/drive/MyDrive/ColabNotebooks/unidata_sub/test_v1.parquet', index = False)

In [ ]:
test_df.to_parquet('/content/drive/MyDrive/ColabNotebooks/splitted_unidata_sub/test_v2.parquet', index = False)

# align subsession to 0 -> 64

In [3]:
def substract_redudant_session(df, test = False):
  min_sub = df.loc[:, ['session','subsession']].groupby('session').min()
  df = df.merge(min_sub, on = 'session', how = 'left')
  df['subsession'] = df['subsession_x'] - df['subsession_y']
  if not test:
    assert df['subsession'].max() == max_session - 1
  assert df['subsession'].min() == 0
  del df['subsession_x'], df['subsession_y']
  return df


In [4]:
train_df = substract_redudant_session(train_df)
train_df

,session,aid,ts,type,diff,subsession
0,0,1517085,1659304800,0,0.0,0
1,0,1563459,1659304904,0,104.0,0
2,0,1309446,1659367439,0,62535.0,1
3,0,16246,1659367719,0,280.0,1
4,0,1781822,1659367871,0,152.0,1
...,...,...,...,...,...,...
196333129,12899776,1737908,1661723987,0,15.0,0
196333130,12899777,384045,1661723976,0,0.0,0
196333131,12899777,384045,1661723986,0,10.0,0
196333132,12899778,561560,1661723983,0,0.0,0


In [5]:
test_df = substract_redudant_session(test_df, test = True)
test_df

,session,aid,ts,type,diff,subsession
0,12899779,59625,1661724000,0,0.0,0
1,12899780,1142000,1661724000,0,0.0,0
2,12899780,582732,1661724058,0,58.0,0
3,12899780,973453,1661724109,0,51.0,0
4,12899780,736515,1661724136,0,27.0,0
...,...,...,...,...,...,...
6928118,14571577,1141710,1662328774,0,0.0,0
6928119,14571578,519105,1662328775,0,0.0,0
6928120,14571579,739876,1662328775,0,0.0,0
6928121,14571580,202353,1662328781,0,0.0,0


# save vprops

In [6]:
def vprops(df, path):
  vprop = df.loc[:, ['session','subsession', 'ts']].groupby('session').agg({
      'ts': ['min', 'max', 'count'],
      'subsession': 'max'
  })
  vprop['consistency'] = vprop.iloc[:, 1] - vprop.iloc[:, 0]
  vprop.reset_index().to_parquet(path, index = False)


In [ ]:
vprops(train_df, '/content/drive/MyDrive/ColabNotebooks/splitted_unidata_sub/train_vprops_v1.parquet')

In [ ]:
vprops(test_df, '/content/drive/MyDrive/ColabNotebooks/splitted_unidata_sub/test_vprops_v1.parquet')

In [7]:
vprops(train_df, '/content/drive/MyDrive/ColabNotebooks/unidata_sub/train_vprops_v1.parquet')

In [8]:
vprops(test_df, '/content/drive/MyDrive/ColabNotebooks/unidata_sub/test_vprops_v1.parquet')

# transform

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
TRAIN_TS_MIN = 1659304800
def transform_df(df):
  df['ts'] = df['ts'] - TRAIN_TS_MIN
  df['clicks'] = np.where(df['type'] == 0, 1, 0)
  df['carts'] = np.where(df['type'] == 1, 1, 0)
  df['orders'] = np.where(df['type'] == 2, 1, 0)
  assert df['clicks'].sum() + df['carts'].sum() + df['orders'].sum() == df.shape[0]
  del df['type'], df['diff']




In [ ]:
transform_df(train_df)
train_df

,session,aid,ts,subsession,clicks,carts,orders
0,0,1517085,0,0,1,0,0
1,0,1563459,104,0,1,0,0
2,0,1309446,62639,1,1,0,0
3,0,16246,62919,1,1,0,0
4,0,1781822,63071,1,1,0,0
...,...,...,...,...,...,...,...
196333129,12899776,1737908,2419187,0,1,0,0
196333130,12899777,384045,2419176,0,1,0,0
196333131,12899777,384045,2419186,0,1,0,0
196333132,12899778,561560,2419183,0,1,0,0


In [ ]:
transform_df(test_df)
test_df

,session,aid,ts,subsession,clicks,carts,orders
0,12899779,59625,2419200,0,1,0,0
1,12899780,1142000,2419200,0,1,0,0
2,12899780,582732,2419258,0,1,0,0
3,12899780,973453,2419309,0,1,0,0
4,12899780,736515,2419336,0,1,0,0
...,...,...,...,...,...,...,...
6928118,14571577,1141710,3023974,0,1,0,0
6928119,14571578,519105,3023975,0,1,0,0
6928120,14571579,739876,3023975,0,1,0,0
6928121,14571580,202353,3023981,0,1,0,0


# Group to edges

In [ ]:

def encode(x):
  return np.sum(2 ** np.unique(x))

In [ ]:
from tqdm.notebook import tqdm
import numpy as np
step = train_df.shape[0] // 10
start = [i * step for i in range(10)]
end = [i + step for i in start]
end[-1] = train_df.shape[0] 

for i in tqdm(range(len(start))):

  train_df.iloc[start[i] : end[i]].groupby(['session', 'aid']).agg({ 
      'ts': ['last','first'],
      'clicks': 'sum',
      'carts': 'sum',
      'orders': 'sum',
      'subsession': encode
  }).reset_index().to_parquet(f'/content/drive/MyDrive/ColabNotebooks/unidata_sub/edges_sub_frag_{i}_v1.parquet', index = False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
from tqdm.notebook import tqdm
import numpy as np
step = train_df.shape[0] // 10
start = [i * step for i in range(10)]
end = [i + step for i in start]
end[-1] = train_df.shape[0] 

for i in tqdm(range(len(start))):

  train_df.iloc[start[i] : end[i]].groupby(['session', 'aid']).agg({ 
      'ts': ['last','first'],
      'clicks': 'sum',
      'carts': 'sum',
      'orders': 'sum',
      'subsession': encode
  }).reset_index().to_parquet(f'/content/drive/MyDrive/ColabNotebooks/splitted_unidata_sub/edges_sub_frag_{i}_v2.parquet', index = False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
import numpy as np
test_df.groupby(['session', 'aid']).agg({ 
    'ts': ['last','first'],
    'clicks': 'sum',
    'carts': 'sum',
    'orders': 'sum',
    'subsession': encode
}).reset_index().to_parquet('/content/drive/MyDrive/ColabNotebooks/unidata_sub/edges_test_sub_v1.parquet', index = False)

In [ ]:
import numpy as np
test_df.groupby(['session', 'aid']).agg({ 
    'ts': ['last','first'],
    'clicks': 'sum',
    'carts': 'sum',
    'orders': 'sum',
    'subsession': encode
}).reset_index().to_parquet('/content/drive/MyDrive/ColabNotebooks/splitted_unidata_sub/edges_test_sub_v2.parquet', index = False)

ITEM FEATURES

In [3]:
import numpy as np
def recent_7days(df):
    unique_aids = np.unique(df.aid.values)
    df['date'] = pd.to_datetime(df.ts * 1e9)#7 recent days
    df['dayofyear'] = df.date.dt.dayofyear
    
    dayofyear_max = df['dayofyear'].max()
    dayofyear_min = dayofyear_max - 6
    
    ts_max = df.ts.max()
    ts_min = (df.date.max() - pd.Timedelta(days = 7)).timestamp()
    df = df[(df.dayofyear >= dayofyear_min) & (df.dayofyear <= dayofyear_max)]
    
    df['dayofyear_type'] = df.dayofyear * 10 + df.type
    item_agg = pd.DataFrame(df[['aid', 'dayofyear_type', 'ts']].groupby(['aid', 'dayofyear_type'])['ts'].count()).rename(columns = {'ts': 'count'})


    dates = pd.DataFrame({'dayofyear_type': np.arange(dayofyear_min, dayofyear_max + 1), 'tmp': 1})
    types = pd.DataFrame({'types': [0,1,2], 'tmp' : 1})
    aids = pd.DataFrame({'aid': np.unique(df.aid), 'tmp': 1})
    dates = dates.merge(types, on = 'tmp')

    dates = aids.merge(dates, on = 'tmp')
    del dates['tmp']
    dates['dayofyear_type'] = dates['dayofyear_type'] * 10 + dates['types']
    del dates['types']
    dates['count'] = np.nan
    dates = dates.groupby(['aid', 'dayofyear_type']).last()


    cols = []
    for i in range(7,0,-1):
        for j in range(3):
            cols.append(f'recent_day{i}_type{j}')
 
    
    item_agg = dates.combine_first(item_agg).fillna(0).reset_index()
    item_agg['count'] = item_agg['count'].values.astype(int)
    recent_7days = pd.DataFrame(data = item_agg[['aid', 'count']].groupby('aid')['count'].apply(list).tolist(), columns = [col for col in cols])
    
    
    item_agg = pd.concat((pd.DataFrame({
        'aid': np.unique(item_agg.aid)
    }), recent_7days), axis = 1)
    
    
    item_agg = pd.DataFrame({'aid': unique_aids, **dict(
        [(col, np.nan) for col in cols], 
    )}).set_index('aid').combine_first(item_agg.set_index('aid')).fillna(0)
    return item_agg.reset_index()

In [4]:
item_agg = recent_7days(train_df)

/tmp/ipykernel_155627/436451667.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayofyear_type'] = df.dayofyear * 10 + df.type


In [5]:
item_agg

,aid,recent_day7_type0,recent_day7_type1,recent_day7_type2,recent_day6_type0,recent_day6_type1,recent_day6_type2,recent_day5_type0,recent_day5_type1,recent_day5_type2,...,recent_day4_type2,recent_day3_type0,recent_day3_type1,recent_day3_type2,recent_day2_type0,recent_day2_type1,recent_day2_type2,recent_day1_type0,recent_day1_type1,recent_day1_type2
0,0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,206.0,18.0,5.0,99.0,11.0,0.0,13.0,0.0,0.0,...,3.0,211.0,27.0,6.0,228.0,19.0,2.0,335.0,32.0,7.0
4,4,8.0,1.0,0.0,12.0,1.0,0.0,7.0,0.0,0.0,...,0.0,8.0,1.0,0.0,10.0,0.0,0.0,14.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855598,1855598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1855599,1855599,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1855600,1855600,2.0,0.0,0.0,4.0,0.0,0.0,3.0,2.0,0.0,...,0.0,4.0,1.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0
1855601,1855601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
item_agg.to_parquet("/home/anhphantq/unidata_sub/item_props_v1.parquet")

In [ ]:
item_agg.to_parquet("/home/anhphantq/splitted_unidata_sub/item_props_v1.parquet")